# Growth rates and fluxes

One of the major applications of micom is to identify growth rates and fluxes for a given community. We will return to our *E. coli* toy example.

In [1]:
from micom import Community, data

tax = data.test_taxonomy()
com = Community(tax)

100%|██████████| 5/5 [00:00<00:00,  6.51models/s]


## FBA

If one is only interested in the community growth rate $\mu_c$ we can use normal FBA to optimize this growth rate. By default micom assigns the community growth rate as the objective for a community model. 

In [3]:
print(com.objective.expression)
print(com.optimize())

1.0*community_objective
<CommunitySolution 0.874 at 0x7f46791417f0>


In [7]:
com.optcom(strategy="lmoma").members

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Escherichia_coli_1,0.2,1.064146e+00,95,72
Escherichia_coli_2,0.2,3.387926e-01,95,72
Escherichia_coli_3,0.2,1.775673e+00,95,72
Escherichia_coli_4,0.2,-7.896792e-32,95,72
Escherichia_coli_5,0.2,1.190997e+00,95,72
medium,NaN,NaN,20,20
